In [7]:
import cv2
import os
import numpy as np
import pandas as pd
import mediapipe as mp

In [8]:
mp_drawing = mp.solutions.drawing_utils
mp_hands   = mp.solutions.hands

In [9]:
# Create a function to setup the directories we'll be storing our images
def makedir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        return None
    else:
        pass

In [10]:
num_classes=5
num_train_images = 200
num_test_images = 50

In [11]:

train = np.random.randn(num_classes,num_train_images,64,64)
test  = np.random.randn(num_classes,num_test_images,64,64)


cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.5) as hands:
    for i in range(0,num_classes):
        
        if i == 0:
            while cv2.waitKey(1) != 13:
                ret, frame = cap.read()
                frame = cv2.flip(frame, 1)

                #define region of interest
                roi = frame[150:400, 320:570]
                roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                roi.flags.writeable= False
                results = hands.process(roi)
                roi.flags.writeable= True
                if results.multi_hand_landmarks:
                    for num,hand in enumerate(results.multi_hand_landmarks):
                        zeros=np.zeros(roi.shape)
                        mp_drawing.draw_landmarks(zeros,hand,mp_hands.HAND_CONNECTIONS)
                        cv2.imshow('hand',zeros)


                cv2.rectangle(frame, (320, 150), (570, 400), (255,0,0), 5)
                cv2.putText(frame, "Hit Enter to Record when ready", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
                cv2.imshow('frame', frame)
                
        image_count = 0

        while(cv2.waitKey(1) != 13 and image_count!=num_train_images):

            ret, frame = cap.read()
            frame = cv2.flip(frame, 1)

            #define region of interest
            roi = frame[150:400, 320:570]
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            roi.flags.writeable= False
            results = hands.process(roi)
            roi.flags.writeable= True

            cv2.rectangle(frame, (320, 150), (570, 400), (255,0,0), 5)
            cv2.putText(frame, "Recording the "+str(i+1)+" gesture - Train", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
            cv2.putText(frame, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
            cv2.imshow('frame', frame)
            
            gesture_dir = './handgestures/train/'+str(i)+'/'
            makedir(gesture_dir)
            
            if results.multi_hand_landmarks:
                    for num,hand in enumerate(results.multi_hand_landmarks):
                        zeros=np.zeros(roi.shape)
                        mp_drawing.draw_landmarks(zeros,hand,mp_hands.HAND_CONNECTIONS)
                        cv2.imshow('hand',zeros)
                        zeros = cv2.resize(zeros, (64, 64))#, interpolation = cv2.INTER_AREA)
                        cv2.imwrite(gesture_dir + str(image_count) + ".jpg", zeros)
                        train[i][image_count] = cv2.imread(gesture_dir + str(image_count) + ".jpg",cv2.IMREAD_GRAYSCALE)
                        image_count+=1
                        
                        

        image_count = 0
        while(cv2.waitKey(1) != 13 and image_count!=num_test_images):

            ret, frame = cap.read()
            frame = cv2.flip(frame, 1)
            #define region of interest
            roi = frame[150:400, 320:570]
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            roi.flags.writeable= False
            results = hands.process(roi)
            roi.flags.writeable= True
            
            cv2.rectangle(frame, (320, 150), (570, 400), (255,0,0), 5)
            cv2.putText(frame, "Recording the "+str(i+1)+" gesture - Test", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
            cv2.putText(frame, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
            cv2.imshow('frame', frame)
            gesture_dir = './handgestures/test/'+str(i)+'/'
            makedir(gesture_dir)
            
            if results.multi_hand_landmarks:
                    for num,hand in enumerate(results.multi_hand_landmarks):
                        zeros=np.zeros(roi.shape)
                        mp_drawing.draw_landmarks(zeros,hand,mp_hands.HAND_CONNECTIONS)
                        cv2.imshow('hand',zeros)
                        zeros = cv2.resize(zeros, (64, 64))
                        cv2.imwrite(gesture_dir + str(image_count) + ".jpg", zeros)
                        test[i][image_count] = cv2.imread(gesture_dir + str(image_count) + ".jpg",cv2.IMREAD_GRAYSCALE)
                        image_count+=1

            
            
        image_count = 0
        if i != num_classes-1:
            while(cv2.waitKey(1) != 13):
                ret, frame = cap.read()
                frame = cv2.flip(frame, 1)

                #define region of interest
                roi = frame[150:400, 320:570]
                roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                roi.flags.writeable= False
                results = hands.process(roi)
                roi.flags.writeable= True
                if results.multi_hand_landmarks:
                    for num,hand in enumerate(results.multi_hand_landmarks):
                        zeros=np.zeros(roi.shape)
                        mp_drawing.draw_landmarks(zeros,hand,mp_hands.HAND_CONNECTIONS)
                        cv2.imshow('hand',zeros)

                cv2.rectangle(frame, (320, 150), (570, 400), (255,0,0), 5)
                cv2.putText(frame, "Click enter to continue ", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
                cv2.imshow('frame', frame)




    
while cv2.waitKey(1) != 13:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    cv2.putText(frame, "Hit Enter to Exit", (100 , 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    cv2.imshow('frame', frame)

cap.release()
cv2.destroyAllWindows() 

In [12]:
print(train.shape)
print(test.shape)

(5, 200, 64, 64)
(5, 50, 64, 64)


In [13]:
train_data=np.random.randn(num_classes*num_train_images,4096)
test_data=np.random.randn(num_classes*num_test_images,4096)
label_train=[]
label_test=[]
for i in range(0,num_classes):
    for j in range(0,num_train_images):
        label_train.append(i)
for i in range(0,num_classes):
    for j in range(0,num_test_images):
        label_test.append(i)

In [14]:
for c in range(0,num_classes):
    train_data[c*num_train_images:num_train_images*(c+1)] = np.array([i.flatten() for i in train[c]])
    test_data[c*num_test_images:num_test_images*(c+1)] = np.array([i.flatten() for i in test[c]])

In [15]:
columns=[]
for i in range(0,4096):
    columns.append('pixel'+str(i))

In [16]:
dataframe_train = pd.DataFrame(train_data,index=None,columns=columns)
dataframe_test=pd.DataFrame(test_data,index=None,columns=columns)
dataframe_train.insert(0, 'label', label_train)
dataframe_test.insert(0, 'label', label_test)

In [17]:
dataframe_train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel4086,pixel4087,pixel4088,pixel4089,pixel4090,pixel4091,pixel4092,pixel4093,pixel4094,pixel4095
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
dataframe_test

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel4086,pixel4087,pixel4088,pixel4089,pixel4090,pixel4091,pixel4092,pixel4093,pixel4094,pixel4095
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
246,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
247,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
248,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
dataframe_train.to_csv(r'train_data_12.csv',index=False)
dataframe_test.to_csv(r'test_data_12.csv',index=False)